In [ ]:
# -----------------------------
# 1️⃣ Import libraries
# -----------------------------
import pandas as pd
from langchain_core.documents import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate

# -----------------------------
# 2️⃣ Load CSV
# -----------------------------
csv_path = "./assets/filtered_data.csv"

print(f"📁 Loading CSV file: {csv_path}")
df = pd.read_csv(csv_path)

print(f"📊 Rows: {len(df)}, Columns: {len(df.columns)}")

# -----------------------------
# 3️⃣ Create Documents (handle NaN)
# -----------------------------
documents = []
for idx, row in df.iterrows():
    # Replace NaN with "N/A"
    values = [str(v) if pd.notna(v) else "N/A" for v in row.values]
    content = " | ".join(values)
    documents.append(
        Document(
            page_content=content,
            metadata={"row_id": idx}
        )
    )

print(f"📚 Created {len(documents)} documents")

# -----------------------------
# 4️⃣ Embedding model
# -----------------------------
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# -----------------------------
# 5️⃣ Create FAISS vector store
# -----------------------------
vectorstore = FAISS.from_texts(
    texts=[doc.page_content for doc in documents],
    embedding=embeddings,
    metadatas=[doc.metadata for doc in documents]
)
print("✅ Vector store created successfully")

# -----------------------------
# 6️⃣ Create Retriever
# -----------------------------
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# -----------------------------
# 7️⃣ Setup LLM
# -----------------------------
llm = Ollama(model="llama3.2:1b")

# -----------------------------
# 8️⃣ Define Prompt Template
# -----------------------------
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an assistant that answers questions using ONLY the CSV data.

CSV DATA:
{context}

Question:
{question}

If the answer is not in the CSV, say:
"I cannot find this information in the CSV."
"""
)

# -----------------------------
# 9️⃣ Define RAG function
# -----------------------------
def ask(question: str):
    # Use get_relevant_texts to avoid AttributeError
    docs = retriever(question)  # bu bir liste döndürüyor
    context = "\n".join(d.page_content for d in docs)
    return llm.invoke(prompt.format(context=context, question=question))

# ------------------------


📁 Loading CSV file: assets/filtered_data.csv
📊 Rows: 2302, Columns: 9
📚 Created 2302 documents
✅ Vector store created successfully


In [29]:
# Use the vectorstore directly
def ask(question: str):
    # FAISS object has a method: similarity_search
    docs = vectorstore.similarity_search(question, k=5)  # top 5 most similar docs
    context = "\n".join(d.page_content for d in docs)
    return llm.invoke(prompt.format(context=context, question=question))


In [30]:
# Test RAG system
answer = ask("What information is stored in this dataset?")
print(f"Answer:\n{answer}")


Answer:
The data type of each column can be determined by examining their names and values.

*   Natural: A variable that can hold any numeric value.
*   Storm: A categorical variable with no numeric or textual representation.
*   Sint Maarten (Dutch part): A categorical variable with two possible values.
*   SXM: A variable containing a single value, but it's not clear if this is an index, category, or something else. However, I'll consider it as a categorical variable for the purpose of this analysis.
*   4 and 32 are numerical values that can hold any type of numeric data.
*   Flood: Another categorical variable with no numeric representation.

So, this dataset appears to contain information about natural phenomena (storms, floods), volcanic activity, island locations, and population statistics.


In [31]:
answer = ask("Which country suffered the most damage?")
print(f"Answer:\n{answer}")


Answer:
To determine which country suffered the most damage, I will calculate the total "Damage" for each country based on the given data.

First, I'll sum up all the values for "Natural" in the columns with unique countries:

- India: 226806 + 197798 = 424604
- Brazil: 197798 + 17897 = 216985
- China: 99168 + 1018663 = 1029921
 

Now, I'll sum up all the values for "Mass movement (wet)" in the columns with unique countries:

- India: 8569.902547 + 3529081.3 = 3604170.802847
- Brazil: 2824.715413 + 317756.5 = 306080.215913
- China: 1053.112314 + 8474922.7 = 8485766.923

Next, I'll compare the total "Damage" for each country to find out which one suffered the most damage:

India had the highest total damage at 424604 units.
Brazil and China also had a significant amount of damage but are tied as they have higher totals than India, with Brazil having 3604170.80 more units in damages than India.

Therefore, I cannot find this information in the CSV.


In [35]:
answer = ask("How much damage did Switzerland suffer?")
print(f"Answer:\n{answer}")


Answer:
To answer this question, we need to look for a column with 'Switzerland' or any reference to Switzerland.

Looking at the data:

- Natural | Glacial lake outburst flood | India | IND | 234 | 226806 | 2021 | 2238.127142 | N/A
- Mass movement (wet) | Indonesia | Sierra Leone | SLE | 1102 | 35818 | 2017 | 484.4561288 | 1120.0

There is no 'Switzerland' or any direct reference to Switzerland in the provided CSV data.

Therefore, I cannot find this information in the CSV.


In [36]:
answer = ask("Summarize the total damage for all countries.")
print(f"Answer:\n{answer}")

Answer:
SELECT SUM(Total) FROM Natural
